In [6]:
from langchain_ollama import ChatOllama
from rich import print as pp

In [2]:
model = ChatOllama(model="llama3.2:1b")

### With stream 

In [3]:
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

The| color| of| the| sky| can| vary| depending| on| the| time| of| day|,| atmospheric| conditions|,| and| other| factors|.| At| sunrise| and| sunset|,| the| sky| can| take| on| a| range| of| colors|,| from| pink| to| orange| to| purple|,| due| to| the| scattering| of| light| by| atmospheric| particles|.

|During| the| daytime|,| when| the| sun| is| overhead|,| the| sky| typically| appears| blue| to| our| eyes|.| This| is| because| blue| light| is| scattered| in| all| directions| by| the| tiny| molecules| of| gases| in| the| Earth|'s| atmosphere|,| such| as| nitrogen| and| oxygen|.

|However|,| if| there| are| clouds| or| other| objects| in| the| air|,| they| can| scatter| light| in| different| ways|,| which| can| cause| the| sky| to| take| on| a| range| of| colors|,| including| gray|,| white|,| or| even| red|.||

### with astream

In [4]:
chunks = []
async for chunk in model.astream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

The| color| of| the| sky| can| vary| depending| on| the| time| of| day|,| atmospheric| conditions|,| and| other| factors|.| During| the| daytime|,| when| the| sun| is| overhead|,| the| sky| typically| appears| blue| due| to| a| phenomenon| called| Ray|leigh| scattering|,| in| which| shorter| wavelengths| of| light| (|like| blue| and| violet|)| are| scattered| more| than| longer| wavelengths| (|like| red| and| orange|)| by| the| tiny| molecules| of| gases| in| the| atmosphere|.

|However|,| at| sunrise| and| sunset|,| the| sky| can| take| on| a| range| of| colors|,| from| pink| and| redd|ish| to| orange| and| purple|,| due| to| the| scattering| of| light| by| atmospheric| particles| and| dust|.| This| is| known| as| M|ie| scattering|.

|In| general|,| though|,| the| sky| is| usually| blue| during| the| daytime|.||

In [7]:
pp(chunks[0])

AIMessageChunk(
    content='The',
    additional_kwargs={},
    response_metadata={},
    id='run-accac619-e706-49b0-8ce1-b46d600d2535'
)

### Chains

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

Why| did| the| par|rot| go| to| the| doctor|?

|Because| it| had| a| f|owl| temper|.||

### Working with Input Streams


In [9]:
from langchain_core.output_parsers import JsonOutputParser

chain = (
    model | JsonOutputParser()
)  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models
async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 67}]}
{'countries': [{'name': 'France', 'population': 67.2}]}
{'countries': [{'name': 'France', 'population': 67200000.0}]}
{'countries': [{'name': 'France', 'population': 67200000.0}, {}]}
{'countries': [{'name': 'France', 'population': 67200000.0}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 67200000.0}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 67200000.0}, {'name': 'Spain', 'population': 46}]}
{'countries': [{'name': 'France', 'population': 67200000.0}, {'name': 'Spain', 'population': 46.7}]}
{'countries': [{'name': 'France', 'population': 67200000.0}, {'name': 'Spain', 'population': 46700000.0}]}
{'countries': [{'name': 'France', 'population': 67200000.0}, {'name': 'Spain', 'population': 46700000.0}, {}]}
{'countries': [{'name': 'France', 'population': 67200000.0}, {'name': 'Spain', '

### Non-streaming components

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

chunks = [chunk for chunk in retriever.stream("where did harrison work?")]
chunks

[[Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='harrison likes spicy food')]]